# Binary star evolution and binary black holes

All the relevant information for the project are to be found in the pdf document present in the repo.
Note that you are assigned to project 1 (as the title said).

## Datasets 

Datasets are stored on Google Drive (link and description in the pdf document)

### Contacts

* Michela Mapelli <michela.mapelli@unipd.it>


In [1]:
import pandas as pd

In [2]:
MTCE_002 = pd.read_csv('/Users/tommaso/Desktop/MagistraleI/lab_project/stable_MT_vs_CE/A0.5/MTCE_BBHs_0.02.txt', skiprows=2 , header=0, sep=' ')

In [3]:
MTCE_002

,col.0:ID,col.1:m1ZAMS/Msun,col.2:m2ZAMS/Msun,col.3:m1rem/Msun,col.4:m2rem/Msun,col.5:m1rem+m2rem/Msun,col.6:delay_time/Myr,col.7:sma/Rsun,col.8:ecc,col.9:kick/kms,...,col.12:cmu2,col.13:v1x/kms,col.14:v1y/kms,col.15:v1z/kms,col.16:v2x/kms,col.17:v2y/kms,col.18:v2z/kms,col.19:time_SN1/Myr,col.20:time_SN2/Myr,col.21:CE
0,0_6942,50.9151,50.8356,4.8001,4.7478,9.5479,2233.6292,7.9524,0.273150,59.6905,...,0.9984,148.1000,-18.66000,5.8540,73.530,23.980,13.120,4.945,4.948,True
1,0_80347,57.0253,56.9464,5.5849,5.5325,11.1174,1076.5703,9.0222,0.508320,46.6697,...,0.9997,148.6000,4.51300,0.8901,103.100,18.180,-10.020,4.705,4.707,True
2,0_158611,91.0594,90.7083,8.3787,8.2624,16.6411,6810.4365,14.9810,0.072798,44.9829,...,0.9990,59.5800,-14.96000,-3.5680,46.490,10.340,-10.480,4.068,4.072,True
3,0_211410,117.9396,57.6917,19.3984,11.2422,30.6406,7905.6343,11229.0000,0.999640,0.0000,...,-0.2000,0.2005,0.02591,0.0000,6.455,-1.241,0.110,3.811,4.539,False
4,0_213101,40.2965,40.2439,3.1045,3.0769,6.1814,8703.5479,13.7630,0.705290,57.8357,...,0.8292,137.1000,-10.12000,0.6957,82.740,50.310,-33.850,5.603,5.608,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,4_3761630,111.0482,95.0845,14.3681,8.9333,23.3014,897.0445,11.6070,0.153100,0.0000,...,0.9998,6.8110,0.00000,0.0000,50.200,7.334,-5.943,3.869,4.019,True
270,4_3944011,51.8496,49.5558,5.4166,4.4900,9.9066,11567.6289,13.5580,0.404530,76.3674,...,0.9991,116.1000,-13.43000,10.8600,121.200,47.630,-10.620,4.879,4.998,True
271,4_3968685,91.6674,90.9037,8.6422,8.4091,17.0513,3437.9648,13.3010,0.211730,25.6512,...,0.9997,56.8100,7.14400,-4.9410,59.880,-6.111,7.464,4.060,4.069,True
272,4_3992400,47.7647,45.8455,5.3031,4.4359,9.7390,649.6041,7.1058,0.500040,86.8337,...,0.9995,147.1000,0.99590,27.7800,118.200,16.630,7.461,5.066,5.185,True


In [4]:
MTCE_002.columns

Index(['col.0:ID', 'col.1:m1ZAMS/Msun', 'col.2:m2ZAMS/Msun',
       'col.3:m1rem/Msun', 'col.4:m2rem/Msun', 'col.5:m1rem+m2rem/Msun',
       'col.6:delay_time/Myr', 'col.7:sma/Rsun', 'col.8:ecc', 'col.9:kick/kms',
       'col.10:kick/kms', 'col.11:cmu1', 'col.12:cmu2', 'col.13:v1x/kms',
       'col.14:v1y/kms', 'col.15:v1z/kms', 'col.16:v2x/kms', 'col.17:v2y/kms',
       'col.18:v2z/kms', 'col.19:time_SN1/Myr', 'col.20:time_SN2/Myr',
       'col.21:CE'],
      dtype='object')

In [5]:
MTCE_002 .loc[:,['col.0:ID','col.1:m1ZAMS/Msun','col.3:m1rem/Msun','col.4:m2rem/Msun',  'col.6:delay_time/Myr', 'col.7:sma/Rsun', 'col.8:ecc', 'col.21:CE']]
#games.loc[:,['offenseFormation','personnelO', 'personnelD', 'typeDropback', 'numberOfPassRushers', 'defendersInTheBox']]
#'col.0:ID' , 'col.1:m1ZAMS/Msun', 'col.2:m2ZAMS/Msun', 'col.3:m1rem/Msun', 'col.4:m2rem/Msun', 'col.5:m1rem+m2rem/Msun', 'col.6:delay_time/Myr', 'col.7:sma/Rsun', 'col.8:ecc', 'col.9:kick/kms', 'col.10:kick/kms', 'col.11:cmu1', 'col.12:cmu2', 'col.13:v1x/kms', 'col.14:v1y/kms', 'col.15:v1z/kms', 'col.16:v2x/kms', 'col.17:v2y/kms', 'col.18:v2z/kms', 'col.19:time_SN1/Myr', 'col.20:time_SN2/Myr', 'col.21:CE'

,col.0:ID,col.1:m1ZAMS/Msun,col.3:m1rem/Msun,col.4:m2rem/Msun,col.6:delay_time/Myr,col.7:sma/Rsun,col.8:ecc,col.21:CE
0,0_6942,50.9151,4.8001,4.7478,2233.6292,7.9524,0.273150,True
1,0_80347,57.0253,5.5849,5.5325,1076.5703,9.0222,0.508320,True
2,0_158611,91.0594,8.3787,8.2624,6810.4365,14.9810,0.072798,True
3,0_211410,117.9396,19.3984,11.2422,7905.6343,11229.0000,0.999640,False
4,0_213101,40.2965,3.1045,3.0769,8703.5479,13.7630,0.705290,True
...,...,...,...,...,...,...,...,...
269,4_3761630,111.0482,14.3681,8.9333,897.0445,11.6070,0.153100,True
270,4_3944011,51.8496,5.4166,4.4900,11567.6289,13.5580,0.404530,True
271,4_3968685,91.6674,8.6422,8.4091,3437.9648,13.3010,0.211730,True
272,4_3992400,47.7647,5.3031,4.4359,649.6041,7.1058,0.500040,True


In [6]:
MTCE_002

,col.0:ID,col.1:m1ZAMS/Msun,col.2:m2ZAMS/Msun,col.3:m1rem/Msun,col.4:m2rem/Msun,col.5:m1rem+m2rem/Msun,col.6:delay_time/Myr,col.7:sma/Rsun,col.8:ecc,col.9:kick/kms,...,col.12:cmu2,col.13:v1x/kms,col.14:v1y/kms,col.15:v1z/kms,col.16:v2x/kms,col.17:v2y/kms,col.18:v2z/kms,col.19:time_SN1/Myr,col.20:time_SN2/Myr,col.21:CE
0,0_6942,50.9151,50.8356,4.8001,4.7478,9.5479,2233.6292,7.9524,0.273150,59.6905,...,0.9984,148.1000,-18.66000,5.8540,73.530,23.980,13.120,4.945,4.948,True
1,0_80347,57.0253,56.9464,5.5849,5.5325,11.1174,1076.5703,9.0222,0.508320,46.6697,...,0.9997,148.6000,4.51300,0.8901,103.100,18.180,-10.020,4.705,4.707,True
2,0_158611,91.0594,90.7083,8.3787,8.2624,16.6411,6810.4365,14.9810,0.072798,44.9829,...,0.9990,59.5800,-14.96000,-3.5680,46.490,10.340,-10.480,4.068,4.072,True
3,0_211410,117.9396,57.6917,19.3984,11.2422,30.6406,7905.6343,11229.0000,0.999640,0.0000,...,-0.2000,0.2005,0.02591,0.0000,6.455,-1.241,0.110,3.811,4.539,False
4,0_213101,40.2965,40.2439,3.1045,3.0769,6.1814,8703.5479,13.7630,0.705290,57.8357,...,0.8292,137.1000,-10.12000,0.6957,82.740,50.310,-33.850,5.603,5.608,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,4_3761630,111.0482,95.0845,14.3681,8.9333,23.3014,897.0445,11.6070,0.153100,0.0000,...,0.9998,6.8110,0.00000,0.0000,50.200,7.334,-5.943,3.869,4.019,True
270,4_3944011,51.8496,49.5558,5.4166,4.4900,9.9066,11567.6289,13.5580,0.404530,76.3674,...,0.9991,116.1000,-13.43000,10.8600,121.200,47.630,-10.620,4.879,4.998,True
271,4_3968685,91.6674,90.9037,8.6422,8.4091,17.0513,3437.9648,13.3010,0.211730,25.6512,...,0.9997,56.8100,7.14400,-4.9410,59.880,-6.111,7.464,4.060,4.069,True
272,4_3992400,47.7647,45.8455,5.3031,4.4359,9.7390,649.6041,7.1058,0.500040,86.8337,...,0.9995,147.1000,0.99590,27.7800,118.200,16.630,7.461,5.066,5.185,True


In [10]:
MTCE_002_true = MTCE_002[(MTCE_002['col.21:CE'] == True)] 

In [11]:
MTCE_002_true

,col.0:ID,col.1:m1ZAMS/Msun,col.2:m2ZAMS/Msun,col.3:m1rem/Msun,col.4:m2rem/Msun,col.5:m1rem+m2rem/Msun,col.6:delay_time/Myr,col.7:sma/Rsun,col.8:ecc,col.9:kick/kms,...,col.12:cmu2,col.13:v1x/kms,col.14:v1y/kms,col.15:v1z/kms,col.16:v2x/kms,col.17:v2y/kms,col.18:v2z/kms,col.19:time_SN1/Myr,col.20:time_SN2/Myr,col.21:CE
0,0_6942,50.9151,50.8356,4.8001,4.7478,9.5479,2233.6292,7.9524,0.273150,59.6905,...,0.9984,148.100,-18.6600,5.8540,73.53,23.980,13.120,4.945,4.948,True
1,0_80347,57.0253,56.9464,5.5849,5.5325,11.1174,1076.5703,9.0222,0.508320,46.6697,...,0.9997,148.600,4.5130,0.8901,103.10,18.180,-10.020,4.705,4.707,True
2,0_158611,91.0594,90.7083,8.3787,8.2624,16.6411,6810.4365,14.9810,0.072798,44.9829,...,0.9990,59.580,-14.9600,-3.5680,46.49,10.340,-10.480,4.068,4.072,True
4,0_213101,40.2965,40.2439,3.1045,3.0769,6.1814,8703.5479,13.7630,0.705290,57.8357,...,0.8292,137.100,-10.1200,0.6957,82.74,50.310,-33.850,5.603,5.608,True
5,0_261642,62.6781,58.8130,7.5024,5.9658,13.4682,2895.4182,10.4610,0.163210,53.5918,...,0.9997,77.030,20.7400,-1.7210,69.77,-8.678,-5.010,4.505,4.629,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,4_3761630,111.0482,95.0845,14.3681,8.9333,23.3014,897.0445,11.6070,0.153100,0.0000,...,0.9998,6.811,0.0000,0.0000,50.20,7.334,-5.943,3.869,4.019,True
270,4_3944011,51.8496,49.5558,5.4166,4.4900,9.9066,11567.6289,13.5580,0.404530,76.3674,...,0.9991,116.100,-13.4300,10.8600,121.20,47.630,-10.620,4.879,4.998,True
271,4_3968685,91.6674,90.9037,8.6422,8.4091,17.0513,3437.9648,13.3010,0.211730,25.6512,...,0.9997,56.810,7.1440,-4.9410,59.88,-6.111,7.464,4.060,4.069,True
272,4_3992400,47.7647,45.8455,5.3031,4.4359,9.7390,649.6041,7.1058,0.500040,86.8337,...,0.9995,147.100,0.9959,27.7800,118.20,16.630,7.461,5.066,5.185,True


In [13]:
MTCE_002_false = MTCE_002[(MTCE_002['col.21:CE'] == False)] 
MTCE_002_false

,col.0:ID,col.1:m1ZAMS/Msun,col.2:m2ZAMS/Msun,col.3:m1rem/Msun,col.4:m2rem/Msun,col.5:m1rem+m2rem/Msun,col.6:delay_time/Myr,col.7:sma/Rsun,col.8:ecc,col.9:kick/kms,...,col.12:cmu2,col.13:v1x/kms,col.14:v1y/kms,col.15:v1z/kms,col.16:v2x/kms,col.17:v2y/kms,col.18:v2z/kms,col.19:time_SN1/Myr,col.20:time_SN2/Myr,col.21:CE
3,0_211410,117.9396,57.6917,19.3984,11.2422,30.6406,7905.6343,11229.0,0.99964,0.0000,...,-0.2000,0.2005,0.025910,0.00,6.455,-1.241,0.11000,3.811,4.539,False
109,1_3370919,76.2366,38.3205,10.0905,8.1532,18.2437,5914.8696,5378.2,0.99951,18.8297,...,-0.4844,1.0660,0.336100,-2.54,13.600,-8.536,0.22850,4.225,6.231,False
113,1_3601930,123.0264,39.9761,18.0234,7.6028,25.6262,9398.3252,3408.7,0.99874,0.0000,...,-0.1979,0.3204,-0.001371,0.00,10.970,1.401,-0.28080,3.802,5.894,False
198,3_2982857,132.6673,30.7234,20.7397,5.6742,26.4139,3331.2690,3755.5,0.99926,0.0000,...,-0.9440,0.3128,0.014260,-0.00,10.070,-4.206,0.05832,3.750,6.506,False
